In [99]:
import os 
import pandas as pd 
import numpy as np
from tqdm import tqdm
import re
tqdm.pandas()

In [100]:
pd.options.display.max_rows = 100

In [101]:
os.chdir("..")

In [102]:
from utils.utils_reader import Reader

In [103]:
reader = Reader()

INFO:LOG-READER:Reader Creation ...
INFO:LOG-READER:Reader Created


In [104]:
reader.download()

INFO:LOG-READER:Authentication Kaggle ...
INFO:LOG-READER:Authenticated Kaggle
INFO:LOG-READER:Downloading ...
INFO:LOG-READER:Downloaded


In [105]:
df = pd.read_csv("data/twcs/twcs.csv")

In [106]:
to_take = 10000

In [107]:
df = df.sort_values(by="tweet_id").reset_index(drop=True)

In [108]:
df = df.iloc[:to_take].copy()

In [109]:
tweet_ids = df.tweet_id.values

starter_tweet_ids = df[df.in_response_to_tweet_id.isna()].tweet_id.values

In [110]:
len(starter_tweet_ids)

2202

In [111]:
df["tread"] = np.NaN
df["order"] = np.NaN
df["company"] = np.NAN
df["start"] = np.NaN
df["end"] = np.NaN
df["company_creator"] = np.NaN

In [132]:
class Node:
    order = 0
    def __init__(self, tweet_id):
        row = df[df.tweet_id == tweet_id].to_dict(orient="records")[0]
        df.loc[df.tweet_id == tweet_id, "order" ] = Node.order
        df.loc[df.tweet_id == tweet_id, "tread" ] = NodesList.tread
        Node.order = Node.order +1 
        self.value = row
        if type(row["response_tweet_id"]) == str :
            self.childrens = [int(x) for x in row["response_tweet_id"].split(",")]
            list_to_check = self.childrens.copy()
            for c in list_to_check:
                if c not in tweet_ids:
                    self.childrens.remove(c)
        else:
            self.childrens = []
        self.children_nodes = list()
        self.__go_deeper()
    
    def __go_deeper(self):
        for children in self.childrens:
            if (self.__check_children(children)):
                self.children_nodes.append(self.__go_deeper_by_children(children))

    def __go_deeper_by_children(self , children):
        children_node = Node(children) 
        return children_node
    
    def __check_children(self, children):
        return children in tweet_ids
    
    def explore(self):
        print(self.value)
        for children in self.children_nodes:
            children.explore()
            
    def __repr__(self):
        return "{\n" + f"{self.value},\n {self.childrens}" + "".join([repr(s) for s in self.children_nodes]) + "\n}"
    
    def __str__ (self):
        return "{\n" + f"{self.value}" + "\n}"

In [133]:
class NodesList:
    tread = 0
    def __init__(self, list_id):
        i = 0
        self.nodes_list = list()
        for x in starter_tweet_ids: 
            i+=1
            self.nodes_list.append(Node(x))
            NodesList.tread = NodesList.tread +1 
            Node.order = 0
            if i%1000 == 0:
                print(f"{i} / {len(starter_tweet_ids)}")

In [114]:
def decomposition(f):
    res = []
    if len(f.children_nodes) == 0:
        return [f.value]
    else:
        for c in f.children_nodes:
            tmp = [f.value , decomposition(c)]
            res.append(tmp)
    return res

In [115]:
def extract_company(row):
    tags = [ x.replace("@","") for x in re.findall(r'@\w+', row.text)]
    res = np.unique([x for x in tags if not x.isnumeric()])
    if len(res) == 0:
        if row.author_id.isnumeric():
            return np.NaN
        else:
            return row.author_id
    else:
        res = res[0]
        if res == row.author_id:
            return res
        else:
            if (res != row.author_id) & (not row.author_id.isnumeric()):
                return row.author_id
            else:
                return res

In [116]:
def user_company_check(row):
    if row.author_id.isnumeric():
        return False #user
    else:
        return True #company

In [117]:
df["company"] = df.progress_apply(extract_company, axis=1)

100%|██████████████████████████████████████████████████████████████████████████| 10000/10000 [00:01<00:00, 8795.85it/s]


In [118]:
df["start"] = df.in_response_to_tweet_id.isna()

In [119]:
df["end"] = df.response_tweet_id.isna()

In [120]:
df["company"] = df.progress_apply(extract_company, axis=1)

100%|██████████████████████████████████████████████████████████████████████████| 10000/10000 [00:01<00:00, 9152.45it/s]


In [121]:
df["company_creator"] = df.progress_apply(user_company_check, axis=1)

100%|█████████████████████████████████████████████████████████████████████████| 10000/10000 [00:00<00:00, 28792.44it/s]


In [122]:
NodesList.tread = 0

In [123]:
starter_tweet_ids

array([    8,    18,    20, ..., 11760, 11762, 11764], dtype=int64)

In [125]:
dfo = df.copy()

In [134]:
df = dfo.copy()

In [135]:
%%time
nodeslist = NodesList(starter_tweet_ids)

1000 / 2202
2000 / 2202
Wall time: 44.4 s


In [136]:
df.head()

,tweet_id,author_id,inbound,created_at,text,response_tweet_id,in_response_to_tweet_id,tread,order,company,start,end,company_creator
0,1,sprintcare,False,Tue Oct 31 22:10:47 +0000 2017,@115712 I understand. I would like to assist y...,2,3.0,0.0,6.0,sprintcare,False,False,True
1,2,115712,True,Tue Oct 31 22:11:45 +0000 2017,@sprintcare and how do you propose we do that,NaN,1.0,0.0,7.0,sprintcare,False,True,False
2,3,115712,True,Tue Oct 31 22:08:27 +0000 2017,@sprintcare I have sent several private messag...,1,4.0,0.0,5.0,sprintcare,False,False,False
3,4,sprintcare,False,Tue Oct 31 21:54:49 +0000 2017,@115712 Please send us a Private Message so th...,3,5.0,0.0,4.0,sprintcare,False,False,True
4,5,115712,True,Tue Oct 31 21:49:35 +0000 2017,@sprintcare I did.,4,6.0,0.0,3.0,sprintcare,False,False,False


In [196]:
class Node:
    order = 0
    def __init__(self, tweet_id):
        row = df[df.tweet_id == tweet_id].to_dict(orient="records")[0]
        df.loc[df.tweet_id == tweet_id, "order" ] = Node.order
        df.loc[df.tweet_id == tweet_id, "tread" ] = NodesList.tread
        Node.order = Node.order +1 
        if type(row["response_tweet_id"]) == str :
            self.childrens = [int(x) for x in row["response_tweet_id"].split(",") if int(x) in tweet_ids]
        else:
            self.childrens = []
        self.children_nodes = list()
        self.__go_deeper()
    
    def __go_deeper(self):
        for children in self.childrens:
            if (self.__check_children(children)):
                self.children_nodes.append(self.__go_deeper_by_children(children))

    def __go_deeper_by_children(self , children):
        children_node = Node(children) 
        return children_node
    
    def __check_children(self, children):
        return children in tweet_ids
    
    def explore(self):
        print(self.value)
        for children in self.children_nodes:
            children.explore()
            

In [201]:
class NodesList:
    tread = 0
    def __init__(self, list_id):
        i = 0
        self.nodes_list = list()
        for t in tqdm(range(len(starter_tweet_ids))):
            x = starter_tweet_ids[t]
            i+=1
            self.nodes_list.append(Node(x))
            NodesList.tread = NodesList.tread +1 
            Node.order = 0
            if i%1000 == 0:
                print(f"{i} / {len(starter_tweet_ids)}")

In [202]:
df = dfo.copy()

In [219]:
%%time
nodeslist = NodesList(starter_tweet_ids)

 46%|███████████████████████████████████▊                                          | 1012/2202 [00:11<00:15, 75.19it/s]

1000 / 2202


 92%|██████████████████████████████████████████████████████████████████████▌      | 2017/2202 [00:23<00:01, 101.86it/s]

2000 / 2202


100%|██████████████████████████████████████████████████████████████████████████████| 2202/2202 [00:25<00:00, 85.73it/s]

Wall time: 25.7 s


In [176]:
df.head(30)

,tweet_id,author_id,inbound,created_at,text,response_tweet_id,in_response_to_tweet_id,tread,order,company,start,end,company_creator
0,1,sprintcare,False,Tue Oct 31 22:10:47 +0000 2017,@115712 I understand. I would like to assist y...,2,3.0,0.0,NaN,sprintcare,False,False,True
1,2,115712,True,Tue Oct 31 22:11:45 +0000 2017,@sprintcare and how do you propose we do that,NaN,1.0,0.0,NaN,sprintcare,False,True,False
2,3,115712,True,Tue Oct 31 22:08:27 +0000 2017,@sprintcare I have sent several private messag...,1,4.0,0.0,NaN,sprintcare,False,False,False
3,4,sprintcare,False,Tue Oct 31 21:54:49 +0000 2017,@115712 Please send us a Private Message so th...,3,5.0,0.0,NaN,sprintcare,False,False,True
4,5,115712,True,Tue Oct 31 21:49:35 +0000 2017,@sprintcare I did.,4,6.0,0.0,NaN,sprintcare,False,False,False
5,6,sprintcare,False,Tue Oct 31 21:46:24 +0000 2017,@115712 Can you please send us a private messa...,"5,7",8.0,0.0,NaN,sprintcare,False,False,True
6,7,115712,True,Tue Oct 31 21:47:48 +0000 2017,@sprintcare the only way I can get a response ...,NaN,6.0,0.0,NaN,sprintcare,False,True,False
7,8,115712,True,Tue Oct 31 21:45:10 +0000 2017,@sprintcare is the worst customer service,"9,6,10",NaN,0.0,NaN,sprintcare,True,False,False
8,9,sprintcare,False,Tue Oct 31 21:46:14 +0000 2017,@115712 I would love the chance to review the ...,NaN,8.0,0.0,NaN,sprintcare,False,True,True
9,10,sprintcare,False,Tue Oct 31 21:45:59 +0000 2017,@115712 Hello! We never like our customers to ...,NaN,8.0,0.0,NaN,sprintcare,False,True,True


In [216]:
def fNode(tweet_id,tread):
        row = df[df.tweet_id == tweet_id].to_dict(orient="records")[0]
        #df.loc[df.tweet_id == tweet_id, "order" ] = Node.order
        df.loc[df.tweet_id == tweet_id, "tread" ] = tread
        Node.order = Node.order +1 
        if type(row["response_tweet_id"]) == str :
            childrens = [int(x) for x in row["response_tweet_id"].split(",") if int(x) in tweet_ids]
        else:
            childrens = []
        children_nodes = list()
        for children in childrens:
            if (children in tweet_ids):
                fNode(children, tread)
            

In [217]:
tread = 0
def fNodesList(starter_tweet_ids,tread):
        i = 0
        for t in tqdm(range(len(starter_tweet_ids))):
            x = starter_tweet_ids[t]
            fNode(x,tread)
            tread = tread +1 
            

In [218]:
%%time
fNodesList(starter_tweet_ids,0)

100%|██████████████████████████████████████████████████████████████████████████████| 2202/2202 [00:25<00:00, 84.77it/s]

Wall time: 26 s
